# 크롤링

In [ ]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# '공작' 네이버 영화 페이지 url 복사
test_url='https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=153687&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'
resp= requests.get(test_url)
html=BeautifulSoup(resp.content, 'html.parser')

#전체 댓글수 및 페이지 수
result=html.find('div', {'class':'score_total'}).find('strong').findChildren('em')[0].getText()
comments=int(result.replace(',',''))
pages=int(np.ceil(comments/10))

#코멘트 리스트 
cmlst=pd.DataFrame(columns=['time','ID','text','score','like','dislike','watch_movie'])

#데이터 추출 함수 정의
def get_data(url):
    global cmlst
    global review_text
    global nickname
    global created_at
    global like
    global dislike
    global score
    #url 호출
    resp=requests.get(url)
    html=BeautifulSoup(resp.content,'html.parser')
    #평점 영역 추출
    score_result=html.find('div',{'class':'score_result'})
    lis=score_result.findAll('li')
    for li in lis:
        try: 
            nickname = li.findAll('a')[0].find('span').getText()
        except:
            try:
                nickname = li.findAll('a')[1].find('span').getText()
            except:
                pass
            
        try:
            created_at = datetime.strptime(li.find('dt').findAll('em')[-1].getText(), "%Y.%m.%d %H:%M")
            like=li.find('div',{'class':'btn_area'}).findAll('strong')[0].getText()
            dislike=li.find('div',{'class':'btn_area'}).findAll('strong')[1].getText()
        except AttributeError:
            pass
        
        try:
            attr = li.select_one('a')
            review_text = attr['data-src']
        except KeyError:
            review_text = li.find('p').getText().replace('\t',r'').replace('\n',r'').replace('\r',r'').replace('\t',r'').replace('관람객',r'').replace('스포일러가 포함된 감상평입니다. 감상평 보기',r'')
        score=li.find('em').getText()
        watch_movie = li.find('span', {'class':'ico_viewer'})
        cmlst=cmlst.append({'time':created_at,'ID':nickname,'text':review_text,'score':score,'like':like,
                            'dislike':dislike, 'watch_movie': watch_movie and True or False},ignore_index=True)


for i in range(1,pages+1):
    url=test_url+'&page='+str(i)
    get_data(url)

#csv 출력
cmlst.to_csv('데이터/공작.csv',sep=',', na_rep='NaN', index=False, encoding='CP949')

# 전처리

In [ ]:
#csv 불러오기
df = pd.read_csv('데이터/공작.csv', encoding='CP949')

#결측행 제거
df = df.dropna()

# 공백행 제거
df = df.set_index('ID').reset_index()
df['ID'].replace('', np.nan, inplace=True)
df.dropna(subset=['ID'], inplace=True)

# 숫자행 제거 (숫자행 없음)
print(df.text.str.isnumeric().value_counts())
print(df.text.replace('.','').str.isnumeric().value_counts())

# 특수문자, 한글 자음제거
import re
df['text'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s) for s in df['text']]

# 인코딩 오류 수정
df = df.replace("&#34;",'"')
df = df.replace("&#39","'")
df = df.replace("&gt;",">")
df = df.replace("&lt;","<")

# 중복행 제거 (ID, time, text 기준) -> 22,579행
df = df.drop_duplicates(['ID','time','text'])
df = df.set_index('ID').reset_index()

In [ ]:
# 라벨링할 1점대 데이터 중 21% 랜덤추출 + csv 파일로 따로 저장 -> 모든 영화의 랜덤추출본을 concat해서 직접 라벨링 진행
df_1 = df[df['score']==1]
df_random = df_1.sample(frac=0.21)
df_random.to_csv('공조(1점대)_랜덤추출.csv', sep=',', index=False, encoding='cp949')

# 데이터 개요

### 평점 분포 확인

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)


# 관람객 평점 분포 / 네티즌 평점 분포
df_tf = pd.pivot_table(df, index ='score', columns='watch_movie', values = 'text', aggfunc='count')
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
ax1.bar(df_tf.index, df_tf[False])
ax2.bar(df_tf.index, df_tf[True])
ax1.set_title('관람객X')
ax2.set_title('관람객O')

In [ ]:
# 전체 평점 분포
df_all= pd.pivot_table(df, index ='score', values = 'text', aggfunc='count')
plt.bar(df_all.index, df_all.text)
plt.title('전체 리뷰')

### 전체 댓글 워드클라우드 (2글자 이상+불용어)

In [ ]:
import nltk # konlpy, nltk
from konlpy.corpus import kobill
files_ko = kobill.fileids()
from konlpy.tag import Okt
from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image

# 한국어 불용어 리스트 불러오기
lists = open('한국어불용어100.txt', encoding = 'utf-8')
lines = lists.readlines()
stopwords = []
for line in lines:
    a,b,c = line.split()
    stopwords.append(a)

In [ ]:
words =''
for i in df.text:
    words = words + ' ' + i
a = ['영화', '공작', '보고', '정말', '진짜']
stopwords += a

t= Okt()
tokens_ko = t.nouns(words)
tokens_ko
ko = nltk.Text(tokens_ko, name = '전체평점')

ko = [each_word for each_word in ko if each_word not in stopwords]           
ko = [n for n in ko if len(n) > 1]
ko = nltk.Text(ko, name='전체평점')

data = ko.vocab().most_common(300)
wordcloud = WordCloud(font_path='c:\Windows\Fonts\malgun.ttf',
                      background_color='white', relative_scaling=0.2).generate_from_frequencies(dict(data))
plt.figure(figsize=(12,8))
plt.axis('off')
plt.imshow(wordcloud)

In [ ]:
# 50개 단어 빈도수 막대그래프
plt.figure(figsize=(12,6))
ko.plot(50)
plt.show()
ko.vocab()

### 1점대 댓글 워드클라우드 (2글자 이상 + 불용어)

In [ ]:
df_1 =df[df.score == 1] 

# text 추출
words =''
for i in df_1.text:
    words = words + ' ' + i
    
# 토큰화
t= Okt()
tokens_ko = t.nouns(words)
ko = nltk.Text(tokens_ko, name = '전체평점_1점대')
print(len(ko.tokens))
print(len(set(ko.tokens)))

# 불용어 처리
ko = [each_word for each_word in ko if each_word not in stopwords]
ko = [n for n in ko if len(n) > 1]

ko = nltk.Text(ko, name='전체평점_1점대')

# 빈도수 300개 이상 워드클라우드 작성
data = ko.vocab().most_common(300)
wordcloud = WordCloud(font_path='c:\Windows\Fonts\malgun.ttf',
                      background_color='white', relative_scaling=0.2).generate_from_frequencies(dict(data))
plt.figure(figsize=(12,8))
plt.axis('off')
plt.imshow(wordcloud)

In [ ]:
plt.figure(figsize=(12,6))
ko.plot(50)
plt.show()
ko.vocab()

### 10점대 댓글 워드클라우드

In [ ]:
df_10 =df[df.score == 10] 

# text 추출
words =''
for i in df_10.text:
    words = words + ' ' + i
    
# 토큰화
t= Okt()
tokens_ko = t.nouns(words)
ko = nltk.Text(tokens_ko, name = '전체평점_10점대')
print(len(ko.tokens))
print(len(set(ko.tokens)))

# 불용어 처리
ko = [each_word for each_word in ko if each_word not in stopwords]
ko = [n for n in ko if len(n) > 1]

ko = nltk.Text(ko, name='전체평점_10점대')

# 빈도수 300개 이상 워드클라우드 작성
data = ko.vocab().most_common(300)
wordcloud = WordCloud(font_path='c:\Windows\Fonts\malgun.ttf',
                      background_color='white', relative_scaling=0.2).generate_from_frequencies(dict(data))
plt.figure(figsize=(12,8))
plt.axis('off')
plt.imshow(wordcloud)

In [ ]:
# 50개 단어 빈도수 막대그래프
plt.figure(figsize=(12,6))
ko.plot(50)
plt.show()
ko.vocab()